In [1]:
import pandas as pd
import numpy as np
import time, json, gzip, re

In [2]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

## read data

In [ ]:
%%time
with gzip.open('vacancies.json.gzip') as f:
    data = json.load(f)
    
df = pd.DataFrame(data)

## prepr

In [ ]:
df['description'] = df['description'].apply(cleanhtml)
df['key_skills_str'] = df['key_skills'].apply(lambda x: ' '.join([el['name'] for el in x]) if x!=[] else None)
df['driver_license_str'] = df['driver_license_types'].apply(lambda x: ' '.join([el['id'] for el in x]) 
                                                            if x!=[] else None).fillna('no_info')

In [ ]:
currency_df = df[['area.name', 'salary.currency', 'id']]\
                            .groupby(['area.name', 'salary.currency'])\
                            .count()\
                            .sort_values('id', ascending=False)
currency_df['rnk'] = currency_df.groupby('area.name').cumcount()
currency_df = currency_df[currency_df['rnk']==0].reset_index()\
                                                .rename(columns={'salary.currency': 'freq_currency'})\
                                                .drop(['rnk', 'id'], axis=1)

df = df.merge(currency_df, on=['area.name'], how='left')
df['salary.currency'] = df['salary.currency'].fillna(df['freq_currency']).fillna('RUR')
currency_mapper = {'RUR': 1, 'BYR': 29.62, 'KZT': 0.16, 'UAH': 2.72, 
                   'USD': 73, 'UZS': 0.0077, 'EUR': 80, 
                   'KGS': 0.95, 'AZN': 43.41, 'GEL': 23.39}

In [ ]:
df['salary.gross'].value_counts()

In [ ]:
tfidf_cols = ['name', 'description', 'key_skills_str', ]
ohe_cols = ['salary.currency', 'driver_license_str', ]
freq_cat_cols = ['area.name']
make_bins = ['salary_from_rur', 'salary_to_rur']

In [89]:
df['salary.gross'] = df['salary.gross'].fillna(True)
df['salary_from_rur'] = df[['salary.from', 'salary.currency']].apply(lambda x: x[0]*currency_mapper[x[1]], axis=1)
df['salary_to_rur'] = df[['salary.to', 'salary.currency']].apply(lambda x: x[0]*currency_mapper[x[1]], axis=1)

In [87]:
df.head().T

,0,1,2,3,4
id,00000692db4bd3c5f0ba856117517c3f,00000b701779500a5400c614379107b4,000023608df632ba86fe90e59b9db01a,000038d7503dc0fb26ab0b1bb3a735e8,00003be6c4c2d06a20205a05f5de1b56
name,Оператор саll-центра,Бухгалтер по начислению заработной платы,Водитель-международник категории Е,Автор работ по информатике,Врач-терапевт
description,Каждый день миллионы пользователей находят отв...,Обязанности: Полное ведение участка заработно...,Добро пожаловать в дружный коллектив компании ...,"Work5 помогает студентам, когда нужно сделать ...",Обязанности: организация и проведение лечебно...
key_skills,[],[{'name': 'Суммированный учет рабочего времени...,[],"[{'name': 'Английский язык'}, {'name': 'Пользо...",[]
driver_license_types,[],[],[],[],[]
area.name,Нижневартовск,Москва,Лида,Белгород,Анапа
salary.from,NaN,65000,1300,40000,25000
salary.to,NaN,NaN,NaN,NaN,NaN
salary.currency,RUR,RUR,EUR,RUR,RUR
salary.gross,None,False,True,False,True


In [28]:
list(df['area.name'].value_counts().index)

['Москва',
 'Санкт-Петербург',
 'Минск',
 'Екатеринбург',
 'Новосибирск',
 'Краснодар',
 'Нижний Новгород',
 'Казань',
 'Уфа',
 'Самара',
 'Ростов-на-Дону',
 'Воронеж',
 'Красноярск',
 'Алматы',
 'Челябинск',
 'Омск',
 'Пермь',
 'Тюмень',
 'Иркутск',
 'Волгоград',
 'Саратов',
 'Ярославль',
 'Барнаул',
 'Ижевск',
 'Оренбург',
 'Хабаровск',
 'Киев',
 'Тула',
 'Нур-Султан',
 'Тверь',
 'Томск',
 'Белгород',
 'Владивосток',
 'Рязань',
 'Тольятти',
 'Кемерово',
 'Липецк',
 'Владимир',
 'Киров (Кировская область)',
 'Набережные Челны',
 'Салават',
 'Ульяновск',
 'Пенза',
 'Сочи',
 'Чебоксары',
 'Калининград',
 'Калуга',
 'Иваново (Ивановская область)',
 'Брянск',
 'Новокузнецк',
 'Ташкент',
 'Курск',
 'Ставрополь',
 'Вологда',
 'Смоленск',
 'Орел',
 'Подольск (Московская область)',
 'Сургут',
 'Симферополь',
 'Астрахань',
 'Химки',
 'Тамбов',
 'Кострома',
 'Брест',
 'Мытищи',
 'Мурманск',
 'Архангельск',
 'Улан-Удэ',
 'Чита',
 'Магнитогорск',
 'Зеленоград',
 'Череповец',
 'Одинцово',
 'Новоро